In [ ]:
import matplotlib.pyplot as plt

# Assuming offline_losses is a dictionary with logged losses
epochs = list(range(len(offline_losses['batch_losses'])))

plt.figure(figsize=(10, 5))
plt.plot(epochs, offline_losses['batch_losses'], label='Training Loss')
plt.plot(epochs, offline_losses['ar_losses'], label='AR Loss')
if 'clip_losses' in offline_losses:
    plt.plot(epochs, offline_losses['clip_losses'], label='Clip Loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Losses')
plt.legend()
plt.show()

In [6]:
import os
import sys

data_dir = "./COATI/"
dataset_name = "3_Semester/Seminar_Practical Work/Code.nosync/COATI/examples/train_valid_test_guacamol.pkl"
import os

cache_dir, file_name = os.path.split(dataset_name)
print(cache_dir)
print(file_name)
if not os.path.exists(data_dir + file_name):
    print("Dataset file not found")
    sys.exit(1)

3_Semester/Seminar_Practical Work/Code.nosync/COATI/examples
train_valid_test_guacamol.pkl
Dataset file not found


SystemExit: 1

/Users/stefanhangler/opt/anaconda3/envs/DLNN1/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import os
import pickle # Stefan
from torch.utils.data.datapipes.iter import IterableWrapper # Stefan

from coati.common.util import dir_or_file_exists

: 

In [1]:
class COATI_dataset:
    def __init__(
        self,
        cache_dir,
        fields=["smiles", "atoms", "coords"],
        test_split_mode="row",
        test_frac=0.02,  # in percent.
        valid_frac=0.02,  # in percent.
    ):
        self.cache_dir = cache_dir
        self.summary = {"dataset_type": "coati", "fields": fields}
        self.test_frac = test_frac
        self.fields = fields
        self.valid_frac = valid_frac
        assert int(test_frac * 100) >= 0 and int(test_frac * 100) <= 50
        assert int(valid_frac * 100) >= 0 and int(valid_frac * 100) <= 50
        assert int(valid_frac * 100 + test_frac * 100) < 50
        self.test_split_mode = test_split_mode

    def partition_routine(self, row):
        """ """
        # if not "mod_molecule" in row:
        #     tore = ["raw"]
        #     tore.append("train")
        #     return tore
        # else:
        #     tore = ["raw"]

        #     if row["mod_molecule"] % 100 >= int(
        #         (self.test_frac + self.valid_frac) * 100
        #     ):
        #         tore.append("train")
        #     elif row["mod_molecule"] % 100 >= int((self.test_frac * 100)):
        #         tore.append("valid")
        #     else:
        #         tore.append("test")

        #     return tore
        # Stefan --->
        partition = row.get('partition', 'raw')
        return [partition]

    def get_data_pipe(
        self,
        rebuild=False,
        batch_size=32,
        partition: str = "raw",
        required_fields=[],
        distributed_rankmod_total=None,
        distributed_rankmod_rank=1,
        xform_routine=lambda X: X,
    ):
        """
        Look for the cache locally
        then on s3 if it's not available locally
        then return a pipe to the data.
        """
        # print(f"trying to open a {partition} datapipe for...")
        # if (
        #     not dir_or_file_exists(os.path.join(self.cache_dir, S3_PATH, "0.pkl"))
        # ) or rebuild:
        #     makedir(self.cache_dir)
        #     query_yes_no(
        #         f"Will download ~340 GB of data to {self.cache_dir} . This will take a while. Are you sure?"
        #     )
        #     copy_bucket_dir_from_s3(S3_PATH, self.cache_dir)

        # pipe = (
        #     FileLister(
        #         root=os.path.join(self.cache_dir, S3_PATH),
        #         recursive=False,
        #         masks=["*.pkl"],
        #     )
        #     .shuffle()
        #     .open_files(mode="rb")
        #     .unstack_pickles()
        #     .unbatch()
        #     .shuffle(buffer_size=200000)
        # )
        # pipe = pipe.ur_batcher(
        #     batch_size=batch_size,
        #     partition=partition,
        #     xform_routine=xform_routine,
        #     partition_routine=self.partition_routine,
        #     distributed_rankmod_total=distributed_rankmod_total,
        #     distributed_rankmod_rank=distributed_rankmod_rank,
        #     direct_mode=False,
        #     required_fields=self.fields,
        # )
        # return pipe

        print(f"trying to open a {partition} datapipe for...")

        # Path to your preprocessed pickle file
        pickle_file = os.path.join(self.cache_dir, "train_valid_test_guacamol.pkl")

        if not dir_or_file_exists(pickle_file):
            raise FileNotFoundError(f"{pickle_file} does not exist. Please ensure the file is in the correct location.")

        # Load the pickle file
        with open(pickle_file, 'rb') as f:
            data = pickle.load(f)

        # Filter data based on the partition
        filtered_data = [row for row in data if row.get('partition') == partition]

        # Create an IterableWrapper from the filtered data
        pipe = IterableWrapper(filtered_data)

        # Shuffle, batch, and transform the data
        pipe = pipe.shuffle().batch(batch_size).map(xform_routine)

        return pipe
